# Séance 1 :  Deep Learning - Introduction à Pytorch 

Les notebooks sont très largement inspirés des cours de **N. Baskiotis et B. Piwowarski**. Ils peuvent être complétés efficacement par les tutoriels *officiels* présents sur le site de pytorch:
https://pytorch.org/tutorials/

Au niveau de la configuration, toutes les installations doivent fonctionner sur Linux et Mac. Pour windows, ça peut marcher avec Anaconda à jour... Mais il est difficile de récupérer les problèmes.

* Aide à la configuration des machines: [lien](https://dac.lip6.fr/master/environnement-deep/)
* Alternative 1 à Windows: installer Ubuntu sous Windows:  [Ubuntu WSL](https://ubuntu.com/wsl)
* Alternative 2: travailler sur Google Colab (il faut un compte gmail + prendre le temps de comprendre comment accéder à des fichers) [Colab](https://colab.research.google.com)

In [ ]:
import torch
print("La version de torch est : ",torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())

# pour les possesseurs de mac M1 avec la dernière version de pytorch:
print("Le calcul GPU est disponible ? ", torch.backends.mps.is_available())

import matplotlib.pyplot as plt
import numpy as np
import sklearn

# B. Autograd et graphe de calcul
Un élément central de pytorch est le graphe de calcul : lors du calcul d'une variable, l'ensemble des opérations qui ont servies au calcul sont stockées sous la forme d'un graphe acyclique, dit de *calcul*. Les noeuds internes du graphe représentent les opérations, le noeud terminal le résultat et les racines les variables d'entrées. Ce graphe sert en particulier à calculer les dérivées partielles de la sortie par rapport aux variables d'entrées - en utilisant les règles de dérivations chainées des fonctions composées. 
Pour cela, toutes les fonctions disponibles dans pytorch comportent un mécanisme, appelé *autograd* (automatique differentiation), qui permet de calculer les dérivées partielles des opérations. 

## B.1. Différenciation automatique
(De manière simplifiée, pour les détails cf [la documentation](https://pytorch.org/docs/stable/notes/extending.html))

Toute opération sous pytorch hérite de la classe **Function** et doit définir :
* une méthode **forward(\*args)** : passe avant, calcule le résultat de la fonction appliquée aux arguments 
* une méthode **backward(\*args)** : passe arrière, calcule les dérivées partielles par rapport aux entrées. Les arguments de  cette méthode correspondent aux valeurs des dérivées suivantes dans le graphe de calcul. En particulier, il y a autant d'arguments à **backward**  que de sorties pour la méthode **forward** (rétro-propagation : on doit connaître les dérivés qui viennent  en aval du calcul) et autant de sorties que d'arguments dans la méthode **forward** (chaque sortie correspond à  une dérivée partielle par rapport à chaque entrée du module). Le calcul se fait sur les valeurs du dernier appel de **forward**. 

Par exemple, pour la fonction d'addition  **add(x,y)**, **add.forward(x,y)** renverra **x+y** (l'appel de la fonction est équivalent à l'appel de **forward**) et **add.backward(1)** renverra le couple **(1,1)** (la dérivée par rapport à x, et celle par rapport à y) .

En pratique, ce ne sont pas les méthodes de ces fonctions qui sont utilisées, mais des méthodes équivalentes sur les tenseurs. La méthode **backward** d'un tenseur permet de rétro-propager le calcul du gradient sur toutes les variables qui ont servies à son calcul.

La valeur du gradient pour chaque dérivée partielle se trouve dans l'attribut **grad** de la variable concernée. 

Comme c'est un mécanisme lourd, l'autograd n'est pas activé par défaut pour une variable. Afin de l'activer, il faut mettre le flag **requires_grad** de cette variable à vrai. Dès lors, tout calcul qui utilise cette variable sera enregistré dans le graphe de calcul et le gradient sera disponible.


Exemple : 

In [ ]:
a = torch.tensor(1.)
# Par défaut, requires_grad est à False
print("Graphe de calcul ? ",a.requires_grad)
# On peut demander à ce que le graphe de calcul soit retenu
a.requires_grad = True 
# Ou lors de la création du tenseur directement
b = torch.tensor(2.,requires_grad=True)
z = 2*a + b
# Calcul des dérivées partielles par rapport à z
z.backward()
print("Dérivée de z/a : ", a.grad.item()," z/b :", b.grad.item())

# Si on a oublié de demander le graphe de calcul :
a, b = torch.tensor(1.),torch.tensor(2.)
z = 2*a+b
try: # on sait que ça va provoquer une erreur
  z.backward()
except Exception as e: # erreur => simple message
  print("Erreur : ", e)

## B.2. <span class="alert-success">     Exercice :  Utilisation de backward     </div>
* Implémentez (en une ligne) la fonction de coût aux moindres carrés $MSE(\hat{y},y)=\frac{1}{2N} \sum_{i=1}^N\|\hat{y_i}-y_i\|^2$ où $\hat{y},y$ sont deux matrices de taille $N\times d$, et $y_i,\hat{y_i}$ les $i$-èmes vecteurs lignes des matrices.
* Engendrez **y,yhat** deux matrices aléatoires de taille $(1,5)$.
* Calculez **MSE(y,yhat)**
* Calculez à la main le gradient de **MSE** par rapport à **y**, **yhat**
* Calculez grâce à pytorch le gradient de **MSE** par rapport à **y** et **yhat** et vérifier le résultat.
* Appelez une deuxième fois **MSE** sur les mêmes vecteurs et la méthode **backward**. Qu'observez vous pour le gradient ? Comment l'expliquez vous ?

In [ ]:
def MSE(yhat,y):
    # Compléter la fonction 
    ##  TODO 
    pass

y = torch.randn(1,5,requires_grad=True)
yhat = torch.randn(1,5,requires_grad=True)
mse = MSE(yhat,y)
print("MSE :" ,mse)

# 1. retro-propager l'erreur
# 2. afficher le gradient sur les deux vecteurs et comprendre ce qui se passe
# 3. faire une itération supplémentaire et afficher de nouveau

##  TODO 


## B.3. <span class="alert-success"> Exercice :   Régression linéaire en pytorch </span>

* Définissez la fonction **flineaire(x,w,b)** fonction linéaire qui calcule $f(x,w,b)=x.w^t+b$  avec $x\in \mathbb{R}^{{n\times d}},~w\in\mathbb{R}^{1,d}, b\in \mathbb{R}$
* Complétez le code ci-dessous pour réaliser une descente de gradient et apprendre les paramètres optimaux de la regression linéaire : $$w^∗,b^∗=\text{argmin}_{w,b}\frac{1}{N} \sum_{i=1}^N \|f(x^i,w,b)-y^i\|^2$$

Pour tester votre code, utilisez le jeu de données très classique *housing*, le prix des loyers à housing en fonction de caractéristiques socio-économiques des quartiers. Le code ci-dessous permet de les charger.

<span style="color:red"> ATTENTION ! </span>
* pour la mise-à-jour des paramètres, <span style="color:red">vous ne pouvez pas faire directement</span> 
$$w = w-\epsilon*gradient$$ 
(pourquoi ?). Vous devez passer par w.data qui permet de ne pas enregistrer les opérations dans le graphe de calcul (ou utiliser la méthode ```.detach()``` d'une variable qui permet de créer une copie détachée du graphe de calcul). 
* Note: il est aussi possible de faire:
    ```
    with torch.no_grad():
        w -= eps*gradient
    ```
    * Désactivation temporaire du graph de calcul, on manipule les tensors comme des variables classiques
    * ATTENTION à faire des ```-=``` ou ```+=``` => Si vous construisez un nouveau tenseur, il ne se reconnectera pas au graphe de calcul!
* l'algorithme doit converger avec la valeur de epsilon fixée; si ce n'est pas le cas, il y a une erreur (la plupart du temps au niveau du calcul du coût).


In [ ]:
def flineaire(x,w,b):
    ##  TODO 
    pass

## Chargement des données housing (depuis sklearn) et transformation en tensor.
# from sklearn.datasets import load_housing # => removed
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing() ## chargement des données
housing_x = torch.tensor(housing['data'],dtype=torch.float) # penser à typer les données pour éliminer les incertitudes
housing_y = torch.tensor(housing['target'],dtype=torch.float)

print("Nombre d'exemples : ",housing_x.size(0), "Dimension : ",housing_x.size(1))
print("Nom des attributs : ", ", ".join(housing['feature_names']))

print(housing_x[:5])


In [ ]:

EPOCHS = 5000
EPS = 1e-7 # que se passe-t-il lorsqu'on joue avec EPS?

# initialisation aléatoire de w et b
w = torch.randn(1,housing_x.size(1),requires_grad=True)
b =  torch.randn(1,1,requires_grad=True)
loss_h = [] # sauvegarde des valeurs de loss (pas si trivial!)

# boucle de descente de gradient
for i in range(EPOCHS):
    pass
    ## SOLUTION 1: Penser à aller chercher w.data (et sa contrepartie dans le gradient)
    # 1. Construire la loss (+stocker la valeur dans loss_h)
    # 2. Retro-propager
    # 3. MAJ des paramètres
    # 4. Penser à remettre le gradient à 0 (cf exo précédent)
    ##  TODO 

In [ ]:
# une seconde version du même code avec l'environnement torch.no_grad()
# attention, dans ce cas, le += est obligatoire
# code identique (juste changer les 2 lignes de MAJ)

EPOCHS = 5000
EPS = 1e-7
#initialisation aléatoire de w et b
w = torch.randn(1,housing_x.size(1),requires_grad=True)
b =  torch.randn(1,1,requires_grad=True)
loss_h = [] # sauvegarde des valeurs de loss (pas si trivial!)
for i in range(EPOCHS):
    pass
    ## SOLUTION 2: avec torch.no_grad() [toutes les lignes sont identiques, sauf les 2 lignes de MAJ des paramètres]
    ##  TODO 

In [ ]:
# affichage de l'optimisation
plt.figure()
plt.plot(loss_h)
plt.xlabel("epochs")
plt.ylabel("mse loss")


## Optimiseur 
La descente de gradient représente en fait un code standard puisque les dérivées sont calculées automatiquement et que les variables sont idéntifiées.
Pytorch inclut une classe très utile pour la descente de gradient, [torch.optim](https://pytorch.org/docs/stable/optim.html), qui permet :
* d'économiser quelques lignes de codes
* d'automatiser la mise-à-jour des paramètres 
* d'abstraire le type de descente de gradient utilisé (sgd,adam, rmsprop, ...)

Une liste de paramètres à optimiser est passée à l'optimiseur lors de l'initialisation. La méthode **zero_grad()** permet de remettre le gradient à zéro et la méthode **step()** permet de faire une mise-à-jour des paramètres.

Un exemple de code  utilisant l'optimiseur est donné ci-dessous. Testez et comparez les résultats.


In [ ]:
Xdim = housing_x.size(1)

w = torch.randn(1,Xdim,dtype=torch.float,requires_grad=True)
b = torch.randn(1,dtype=torch.float,requires_grad=True)

## on optimise selon w et b.  lr est le pas du gradient
optim = torch.optim.SGD(params=[w,b],lr=EPS) 
for i in range(EPOCHS):
  loss = MSE(flineaire(housing_x,w,b).view(-1,1),housing_y.view(-1,1))
  optim.zero_grad()
  loss.backward()
  optim.step()  
  if i % 100==0:  print(f"iteration : {i}, loss : {loss}")


# Construction du sujet à partir de la correction

In [ ]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###